# Unsupervised Machine Learning - Project1
Objective : Identify Spam and Normal(Ham) emails by subject heading.

Python packages - pandas, numpy, sklearn, word_tokenize

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#This is dataset 
df_data = pd.DataFrame([{'Text':'reading assignment', 'Label':'Ham'},
 {'Text': 'analysis and design assignment', 'Label':'Ham'},
 {'Text': 'hello howare you', 'Label':'Ham'},
 {'Text': 'requirements analysis', 'Label':'Ham'},
 {'Text': 'win free', 'Label':'Spam'},
 {'Text': 'lotto millions', 'Label':'Spam'},
 {'Text': 'win win win millions', 'Label':'Spam'},
 {'Text': 'free offers', 'Label':'Spam'}])
#datset with class Ham
df_data_ham = df_data[df_data.Label == 'Ham']
#datset with class Spam
df_data_spam = df_data[df_data.Label == 'Spam']


In [ ]:
df_data

,Text,Label
0,reading assignment,Ham
1,analysis and design assignment,Ham
2,hello howare you,Ham
3,requirements analysis,Ham
4,win free,Spam
5,lotto millions,Spam
6,win win win millions,Spam
7,free offers,Spam


In [ ]:
#Vectorize the text to create a Term Document Frequency matrix
# all emails: Term Document Frequency
vec = CountVectorizer()
tdf = vec.fit_transform(df_data['Text'])
total_words = len(vec.get_feature_names())
# TDM for ham class
count_vect_ham = CountVectorizer()
tdf_ham = count_vect_ham.fit_transform(df_data_ham['Text'])
tdm_ham = pd.DataFrame(tdf_ham.toarray(), columns=count_vect_ham.get_feature_names())
word_list_ham = count_vect_ham.get_feature_names(); 
tdm_ham_dict = dict(zip(word_list_ham,tdf_ham.toarray().sum(axis=0)))
print(tdm_ham_dict)
# TDM forspamclass
count_vect_spam = CountVectorizer()
tdf_spam = count_vect_spam.fit_transform(df_data_spam['Text'])
tdm_spam = pd.DataFrame(tdf_spam.toarray(), columns=count_vect_spam.get_feature_names())
word_list_spam = count_vect_spam.get_feature_names(); 
tdm_spam_dict = dict(zip(word_list_spam,tdf_spam.toarray().sum(axis=0)))
print(tdm_spam_dict)

{'analysis': 2, 'and': 1, 'assignment': 2, 'design': 1, 'hello': 1, 'howare': 1, 'reading': 1, 'requirements': 1, 'you': 1}
{'free': 2, 'lotto': 1, 'millions': 2, 'offers': 1, 'win': 4}


# Naive Bayes Classifier

In [ ]:
#Compute prior probabilities for Spam and Ham class
prior_prob_spam = df_data_spam.shape[0]/ df_data.shape[0]
prior_prob_ham = df_data_ham.shape[0]/ df_data.shape[0]
#function to calculate posterior probabilities 
def posterior_prob(test_sentence):
 
    prob_spam_given_all_words = 1
    prob_ham_given_all_words = 1
 
    ham_words_count = tdf_ham.toarray().sum()
    spam_words_count = tdf_spam.toarray().sum()
    words = word_tokenize(test_sentence)
    for word in words:
 # P(Word | Spam) with Laplace smoothing
        prob_spam_given_word = ((tdm_spam_dict.get(word, 0) + 1)/(spam_words_count + total_words) )
        prob_spam_given_all_words = prob_spam_given_all_words * prob_spam_given_word
 
 # P(Word | Ham) with Laplace smoothing
    prob_ham_given_word = ((tdm_ham_dict.get(word, 0) + 1)/(ham_words_count + total_words) )
    prob_ham_given_all_words = prob_ham_given_all_words * prob_ham_given_word

# P(Ham | words) = P(Ham) * P(Words | Ham)
    posterior_ham = prior_prob_ham * prob_ham_given_all_words
 # P(Spam | words) = P(Spam) * P(Words | Spam)
    posterior_spam = prior_prob_spam * prob_spam_given_all_words
 
    CLASS = 'Ham' if posterior_ham > posterior_spam else 'Spam'

    prob_ham = posterior_ham /(posterior_spam + posterior_ham)
    prob_spam= posterior_spam /(posterior_spam + posterior_ham)
    return (prob_ham, prob_spam, CLASS)


In [ ]:
#test function - classifying sentence 'Ham' or 'Spam'
prob_ham, prob_spam, c = posterior_prob(test_sentence='win win ')
print("Probability Ham ",prob_ham, " Probability Spam ",prob_spam, "Class", c)

prob_ham, prob_spam, c = posterior_prob(test_sentence='New employee ')
print("Probability Ham ",prob_ham, " Probability Spam ",prob_spam, "Class ", c)

Probability Ham  0.4796003330557868  Probability Spam  0.5203996669442131 Class Spam
Probability Ham  0.9584026622296173  Probability Spam  0.04159733777038269 Class  Ham
